In [1]:
#Import Dependencies 
from bs4 import BeautifulSoup
from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager
import os
import requests
import time
import pandas as pd
import numpy as np

## NASA Mars News

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Neel\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [3]:
#Scrape from site https://redplanetscience.com/ 
#Retrieve data for news_title & news_p
url = "https://redplanetscience.com/"
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [4]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())

<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
  <link href="css/font.css" rel="stylesheet" type="text/css"/>
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
  <title>
   News - Mars Exploration Program
  </title>
 </head>
 <body>
  <div class="col-md-12">
   <div class="row">
    <nav class="navbar navbar-expand-lg navbar-light fixed-top">
     <div class="container-fluid">
      <a class="navbar-brand" href="#">
       <img src="image/nasa.png" width="80"/>
       <span class="logo">
        MA

In [5]:
#Finding the latest news title and latest news teaser body
news_title = soup.find('div', class_='content_title').text
news_paragraph = soup.find('div', class_='article_teaser_body').text

print(news_title)
print('--------------------------------------------------------------------')
print(news_paragraph)

While Stargazing on Mars, NASA's Curiosity Rover Spots Earth and Venus
--------------------------------------------------------------------
This new portrait of the Red Planet's neighbors was taken during a time when there's more dust in the air on Mars.


## JPL Mars Spece Images - Featured Image

In [6]:
#Scrape from site https://spaceimages-mars.com/
url = "https://spaceimages-mars.com/"
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [7]:
# Url of the Featured Image
image_path = soup.find_all('img')[1]["src"]
featured_image_url = url + image_path
featured_image_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

## Mars Facts

In [8]:
#Scrape from site https://galaxyfacts-mars.com/
url = "https://galaxyfacts-mars.com/"
browser.visit(url)
html = browser.html

In [9]:
# Visit the Mars Facts webpage and use Pandas to scrape the table containing facts about the planet including diameter, mass, etc
table = pd.read_html(url)
mars_planet_profile = table[1]
# mars_planet_profile

# Rename Columns
mars_planet_profile.columns = ['', 'Values']
mars_planet_profile.set_index('', inplace = True)
mars_planet_profile

,Values
,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 ( Phobos & Deimos )
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [10]:
# Use Pandas to convert the data to a HTML table string.
mars_planet_profile.to_html('table.html')

## Mars Hemispheres

In [11]:
# Visit the astrogeology site to obtain high-resolution images for each hemisphere of Mars.
url = "https://marshemispheres.com/"
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [12]:
#Save the image URL string for the full resolution hemisphere image and the hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
hemisphere_image_urls=[]
products = soup.find ('div', class_='result-list')
hemispheres = products.find_all('div',{'class':'item'})

for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://marshemispheres.com/" + end_link    
    browser.visit(image_link)
    html_hemispheres = browser.html
    soup=BeautifulSoup(html_hemispheres, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_image_urls.append({"title": title, "img_url": image_url})
        

In [13]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere ', 'img_url': 'images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere ',
  'img_url': 'images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere ',
  'img_url': 'images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere ',
  'img_url': 'images/valles_marineris_enhanced-full.jpg'}]

In [14]:
browser.quit()